In [4]:
import sympy as sym

# Define states, measurements and inputs.
x1, x2, x3 = sym.symbols('x1, x2, x3')
x = sym.Matrix([[x1], [x2], [x3]])

u1 = sym.symbols('u1')
u = sym.Matrix([[u1]])

z1 = sym.symbols('z1')
z = sym.Matrix([[z1]])

w1, w2, w3 = sym.symbols('w1, w2, w3')
w = sym.Matrix([[w1], [w2], [w3]])

v1 = sym.symbols('v1')
v = sym.Matrix([[v1]])

p11, p12, p13, p21, p22, p23, p31, p32, p33 = sym.symbols('p11, p12, p13, p21, p22, p23, p31, p32, p33')
P = sym.Matrix([[p11, p12, p13], [p21, p22, p23], [p31, p32, p33]])

var_x1, var_x2, var_x3 = sym.symbols('var_x1 var_x2 var_x3')
Q = sym.diag(var_x1, var_x2, var_x3)

var_z1 = sym.symbols('var_z1')
R = sym.diag(var_z1)

# Helper variables
dt = sym.Symbol('dt')
c1 = 3.3 / 0x3FF

# Define state model
f = x + (c1 * u1 - x2) * x3 * dt * sym.Matrix([[1], [0], [0]]) + w 
F = f.jacobian(x)
f11, f12, f13, f21, f22, f23, f31, f32, f33 = sym.symbols('f11, f12, f13, f21, f22, f23, f31, f32, f33')
F_sym = sym.Matrix([[f11, f12, f13], [f21, f22, f23], [f31, f32, f33]]) 

B = f.jacobian(u)

h = sym.Matrix([x1]) + v
H = h.jacobian(x)

# Process update
x_pred = f.subs([(w1, 0), (w2, 0), (w3, 0)]) # State prediction export f(x, u, c)
P_pred = F * P * F.transpose() + Q # Covariance prediction export P_pred(x, u, c, Q)

# Measurement update
y = z - h.subs([(v1, 0)]) # Residual
residual = sym.symbols('residual')

S = H * P * H.transpose() + R # Innovation (export S_inv(P, R))
s_inv11 = sym.symbols('s_inv11')
S_inv = sym.Matrix([[s_inv11]]) 

K = P * H.transpose() * S_inv # Kalman gain (export K(P, S_inv))
k1, k2, k3 = sym.symbols('k1, k2, k3')
K_sym = sym.Matrix([[k1], [k2], [k3]]) 

x_est = x + K_sym * residual # Posterior state estimate (export x_est(x, P, z, K))
P_est = (sym.eye(len(x)) - K_sym * H) * P # Posterior covariance estimate (export P_est(P, K))

# Export C-Code
from sympy.utilities.codegen import codegen, get_code_generator, default_datatypes
from sympy.codegen.ast import float32, real

cg = get_code_generator("C", project="clock_sync") #note project must be specified here
cg.printer.type_aliases[real] = float32 #makes the functions correct (cosf instead of cos)
default_datatypes["float"].cname = "float" #makes the data-type correct (float instead of double)

codegen([("predictX", x_pred), ("predictP", P_pred), ("computeResidual", y), ("computeSInverse", S.inv()), ("computeK", K), ("estimateX", x_est), ("estimateP", P_est)], 
        prefix="ClockEstimator", header=False, empty=False, to_files=True, code_gen=cg)




In [14]:
x_pred

Matrix([
[dt*x3*(0.0032258064516129*u1 - x2) + x1],
[                                     x2],
[                                     x3]])

In [3]:
import numpy as np
import scipy.linalg
# Thanks to http://www.mwm.im/lqr-controllers-with-python/
 
def dlqr(A,B,Q,R):
    """Solve the discrete time lqr controller.

    x[k+1] = A x[k] + B u[k]

    cost = sum x[k].T*Q*x[k] + u[k].T*R*u[k]
    """
    #ref Bertsekas, p.151

    #first, try to solve the ricatti equation
    X = np.matrix(scipy.linalg.solve_discrete_are(A, B, Q, R))

    #compute the LQR gain
    K = np.matrix(scipy.linalg.inv(B.T*X*B+R)*(B.T*X*A))

    eigVals, eigVecs = scipy.linalg.eig(A-B*K)

    return K, X, eigVals

In [4]:
import numpy as np
import control 
from control import ctrb, obsv

# Treat x2 and x3 as constants and replace with nominal values
x_ctrl = sym.Matrix([x1])

f_ctrl = sym.Matrix([x_pred[0]])
f_ctrl = f_ctrl.subs([(x2, 1.5), (x3, 5.0), (dt, 1.0)])

A = f_ctrl.jacobian(x_ctrl)
B = f_ctrl.jacobian(u)

A = np.array(A).astype(np.float64)
B = np.array(B).astype(np.float64)

Q = np.zeros((1, 1))
Q[0,0] = 1.0

R = np.zeros((1,1))
R[0,0] = 0.1

# Controllability check
if (np.linalg.matrix_rank(ctrb(A,B)) == len(x_ctrl)):
    print("System (A,B) is controllable.")
else:
    print("System (A,B) is NOT controllable.")
    
# Observability check
if (np.linalg.matrix_rank(obsv(A,Q)) == len(x_ctrl)):
    print("System (A,Q) is observable.")
else:
    print("System (A,Q) is NOT observable.")

Klqr, Xlqr, Elqr = dlqr(A, B, Q, R)
print("Gain: ")
print(Klqr)



System (A,B) is controllable.
System (A,Q) is observable.
Gain: 
[[3.08266065]]


In [38]:
x_pred

Matrix([
[dt*x3*(0.0032258064516129*u1 - x2) + x1],
[                                     x2],
[                                     x3]])